In [1]:
#setup

%load_ext autoreload
%autoreload 2

import random
import json
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt


import sys


# add /Users/ywxiu/jasp-multimodal-rag/src to sys.path
project_root = Path.cwd().parent  # goes from notebooks → jasp-multimodal-rag
src_path = project_root / "src"
sys.path.append(str(src_path))

import retrieval.retrieval as retr  # adjust if needed


2025-11-19 01:23:03.785 | INFO     | retrieval.retrieval:<module>:56 - ✅ Loaded FlagEmbeddingReranker from llama_index.postprocessor.flag_embedding_reranker


In [7]:
#Step 1 – Load test_set
TEST_JSON = Path("/Users/ywxiu/jasp-multimodal-rag/data/test_QA/QA_filled_1.json")

with open(TEST_JSON, "r") as f:
    test_set = json.load(f)


print(list(test_set[0]))
len(test_set)

['id', 'query', 'answerable', 'ground_truth_answer', 'relevant_chunk_ids']


12

In [8]:
#Step 2 – Parameter space for tuning
param_space = {
    "K_BM25":          [5, 10, 20],
    "boost_weight":    [0.0, 1.0, 3.0, 5.0],
    "K_SEMANTIC":      [5, 10, 20],
    "RRF_K":           [10, 60, 120],
    "TOP_AFTER_RRF":   [10, 20, 40],
    "score_threshold": [-3, -1, 0, 1.5, 2.0, 2.5, 3.0],
    "TOP_FINAL":       [1, 3, 5],
}


In [9]:
#Step 3 – Helper to apply a config to the retrieval module
def apply_config(cfg: dict):
    retr.K_BM25        = cfg["K_BM25"]
    retr.BOOST_WEIGHT  = cfg["boost_weight"]
    retr.K_SEMANTIC    = cfg["K_SEMANTIC"]
    retr.RRF_K         = cfg["RRF_K"]
    retr.TOP_AFTER_RRF = cfg["TOP_AFTER_RRF"]
    retr.SCORE_THRESHOLD = cfg["score_threshold"]
    retr.TOP_FINAL     = cfg["TOP_FINAL"]


In [10]:
#Step 4 – Evaluation function (compute H1a, H1b, H2)

def evaluate_config(cfg: dict, test_set: list):
    """Run retrieval for a given config and compute H1a, H1b, H2."""
    apply_config(cfg)

    retrieval_results = []

    for item in test_set:
        q = item["query"]
        qid = item["id"]
        relevant_ids = set(item["relevant_chunk_ids"])

        # Run your pipeline
        results = retr.retrieve_top_k(q, top_k=cfg["TOP_FINAL"])

        retrieved_ids = [
            # try doc_id, fall back to section_id
            (getattr(r, "metadata", {}) or {}).get("doc_id")
            or (getattr(r, "metadata", {}) or {}).get("section_id")
            for r in results
        ]

        success_at_k = any(rid in relevant_ids for rid in retrieved_ids[:cfg["TOP_FINAL"]])
        top1_relevant = retrieved_ids[0] in relevant_ids if retrieved_ids else False

        retrieval_results.append({
            "id": qid,
            "query": q,
            "answerable": item["answerable"],
            "retrieved_ids": retrieved_ids,
            "success_at_k": success_at_k,
            "top1_relevant": top1_relevant,
        })

    df = pd.DataFrame(retrieval_results)
    df_answerable   = df[df["answerable"] == True]
    df_unanswerable = df[df["answerable"] == False]

    H1a = df_answerable["success_at_k"].mean() if not df_answerable.empty else float("nan")
    H1b = df_answerable["top1_relevant"].mean() if not df_answerable.empty else float("nan")

    if not df_unanswerable.empty:
        is_empty = (df_unanswerable["retrieved_ids"].str.len() == 0)
        H2 = is_empty.mean()
    else:
        H2 = float("nan")

    return H1a, H1b, H2


In [ ]:
#Step 5 – Randomly sample configurations and evaluate
def sample_random_config(space: dict) -> dict:
    return {k: random.choice(v) for k, v in space.items()}

N_EXPERIMENTS = 30  # you can make this 30, 50, 100...

results = []

for i in range(N_EXPERIMENTS):
    cfg = sample_random_config(param_space)
    H1a, H1b, H2 = evaluate_config(cfg, test_set)

    row = {**cfg, "H1a": H1a, "H1b": H1b, "H2": H2}
    results.append(row)

    print(f"{i+1}/{N_EXPERIMENTS} | {cfg} | H1a={H1a:.3f}, H1b={H1b:.3f}, H2={H2:.3f}")

df_results = pd.DataFrame(results)
df_results.head()


2025-11-19 01:24:44.256 | INFO     | retrieval.retrieval:retrieve_top_k:370 - 🔍 Starting retrieval pipeline for query: 'How to import data from Excel or CSV into JASP?'
2025-11-19 01:24:44.262 | INFO     | retrieval.retrieval:load_docs_for_bm25:199 - 📄 Loaded 16 chunks from /Users/ywxiu/jasp-multimodal-rag/data/processed/chunks/github_RegressionLinearBayesian.json
2025-11-19 01:24:44.262 | INFO     | retrieval.retrieval:load_docs_for_bm25:199 - 📄 Loaded 33 chunks from /Users/ywxiu/jasp-multimodal-rag/data/processed/chunks/github_RegressionLogistic.json
2025-11-19 01:24:44.263 | INFO     | retrieval.retrieval:load_docs_for_bm25:199 - 📄 Loaded 43 chunks from /Users/ywxiu/jasp-multimodal-rag/data/processed/chunks/video_Introduction_to_JASP_APRaBFC2lEQ_chunks.json
2025-11-19 01:24:44.264 | INFO     | retrieval.retrieval:load_docs_for_bm25:199 - 📄 Loaded 58 chunks from /Users/ywxiu/jasp-multimodal-rag/data/processed/chunks/github_MixedModelsLMM.json
2025-11-19 01:24:44.265 | INFO     | retr

In [ ]:
# Step 6 – Plot 3 figures: H1a, H1b, H2 vs each parameter
param_names = [
    "K_BM25",
    "boost_weight",
    "K_SEMANTIC",
    "RRF_K",
    "TOP_AFTER_RRF",
    "score_threshold",
    "TOP_FINAL",
]

def plot_metric_vs_params(df, metric: str):
    n_params = len(param_names)
    fig, axes = plt.subplots(1, n_params, figsize=(4*n_params, 4), sharey=True)

    if n_params == 1:
        axes = [axes]

    for ax, param in zip(axes, param_names):
        grouped = df.groupby(param)[metric].mean().sort_index()
        ax.plot(grouped.index, grouped.values, marker="o")
        ax.set_title(param)
        ax.set_xlabel(param)
        ax.grid(True, linestyle="--", alpha=0.4)

    axes[0].set_ylabel(metric)
    fig.suptitle(f"{metric} vs retrieval parameters", fontsize=14)
    plt.tight_layout()
    plt.show()

plot_metric_vs_params(df_results, "H1a")
plot_metric_vs_params(df_results, "H1b")
plot_metric_vs_params(df_results, "H2")


In [ ]:
good = df_results[
   # (df_results["H1a"] >= 0.60) &
   # (df_results["H1b"] >= 0.60) &
    (df_results["H2"] >= 0.80)
].sort_values("H1a", ascending=False)

good.head()
